In [1]:
import os
import warnings
from pathlib import Path

import numpy as np
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import load_model, read_sbml_model
from process_bigraph import Process, Composite, ProcessTypes

# from biosimulators_processes import CORE
# from biosimulators_processes.data_model.sed_data_model import MODEL_TYPE
# from biosimulators_processes.viz.plot import plot_time_series, plot_species_distributions_to_gif


CORE = ProcessTypes()

class CobraProcess(Process):
    config_schema = {
        'model': {
            'model_source': 'string'
        },
        'model_name': {
            '_type': 'string',
            '_default': 'FBA'
        }
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        # create "empty" model
        self.model_name = self.config.get('model_name')
        model_file = self.config['model'].get('model_source')
        if model_file is not None:
            data_dir = Path(os.path.dirname(model_file))
            path = data_dir / model_file.split('/')[-1]
            self.model = read_sbml_model(str(path.resolve()))
        else:
            self.model = None
        # self.model = Model(modelname)

    def initial_state(self):
        if self.model is not None:
            solution = self.model.optimize()
        pass 
        
        
    def inputs(self):
        return {
            'reactions': 'tree[string]',
            'floating_species_concentrations': 'tree[float]'
        }
    
    def outputs(self):
        return {
            'substrates': 'tree[float]'
        }
    
    def update(self, state, interval):
        from random import randint
        # parse input state
        substrates_input = state['floating_species_concentrations']
        reactions = state['reactions']
        species_names = list(substrates_input.keys())

        # generate model based on input state data
        model = self.generate_model(reactions, species_names)

        # mapped one to one to species names
        for metabolite in model.metabolites:
            lookup_key = metabolite.id.replace("_", " ")
            # TODO: make this more dynamic
            Km, Vmax = (0.5, randint(1, 3))
            substrate_concentration = substrates_input[lookup_key]
            uptake_rate = Vmax * substrate_concentration / (Km + substrate_concentration)
            # assign uptake rate to each relevant reaction lower bound
            for reaction in metabolite.reactions:
                model.reactions.get_by_id(reaction.id).lower_bound = -uptake_rate

        
        substrate_update = {}
        solution = model.optimize()
        # if solution.status == 'optimal':
        for metabolite in model.metabolites:
            lookup_key = metabolite.id.replace("_", " ")
            for reaction in metabolite.reactions:
                flux = solution.fluxes[reaction.id]
                old_concentration = substrates_input[lookup_key]
                new_concentration = max(old_concentration + flux, 0)  # keep above 0
                substrate_update[lookup_key] = new_concentration - old_concentration
        # else:
            # substrate_update = {metabolite.id.replace("_", " "): 0 for metabolite in model.metabolites}

        return {'substrates': substrate_update}
                
    def generate_model(self, reactions_dict, species_names) -> Model:
        # Create a new COBRA model
        model = Model(self.model_name)
        reactions = list(reactions_dict.values())
        reaction_names = list(reactions_dict.keys())
        
        # Step 1: Create Metabolite objects for all species
        metabolite_dict = {}
        for species in species_names:
            metabolite = Metabolite(
                species.replace(" ", "_").replace('"', ''),  # Ensure the metabolite ID is valid
                name=species,
                compartment='c'  # Assuming all species are in the cytosol
            )
            metabolite_dict[species] = metabolite  # Add to dictionary for easy reference
    
        # Step 2: Create Reaction objects and add Metabolites
        for i, reaction_scheme in enumerate(reactions):
            reaction_name = reaction_names[i]
            reaction_id = reaction_name.replace(" ", "_")
            reaction = Reaction(reaction_id)  # Create a reaction with the ID
            
            # Parse the reaction scheme to identify substrates and products
            scheme_str = reaction_scheme
            reactants, products = scheme_str.split("->")
            
            # Initialize the dictionary for metabolites with their stoichiometry
            metabolites_to_add = {}
    
            # Handle Reactants
            reactant_list = reactants.split(";")
            for reactant in reactant_list:
                reactant = reactant.strip().replace('"', '')  # Clean up string
                if reactant in metabolite_dict:
                    metabolites_to_add[metabolite_dict[reactant]] = -1.0  # Reactants are consumed (-1)
    
            # Handle Products
            product_list = products.split(";")
            for product in product_list:
                product = product.strip().replace('"', '')  # Clean up string
                if product in metabolite_dict:
                    metabolites_to_add[metabolite_dict[product]] = 1.0  # Products are produced (+1)
    
            # Add metabolites to the reaction
            reaction.add_metabolites(metabolites_to_add)
    
            # Add the reaction to the model
            model.add_reactions([reaction])
    
        return model


In [2]:
from typing import Dict, Union, Optional, List, Any
from datetime import datetime
import json

import numpy as np
from COPASI import CDataModel
from numpy import ndarray, dtype, array, append as npAppend
from pandas import DataFrame
from basico import (
    load_model,
    get_species,
    get_parameters,
    get_reactions,
    set_species,
    run_time_course_with_output,
    run_time_course,
    get_compartments,
    new_model,
    set_reaction_parameters,
    add_reaction,
    set_reaction,
    set_parameters,
    add_parameter
)


class CopasiProcess(Process):
    config_schema = {
        'model': {
            'model_source': 'string'
        },
        'method': {
            '_type': 'string',
            '_default': 'deterministic'
        }
    }

    def __init__(self,
                 config: Dict[str, Union[str, Dict[str, str], Dict[str, Optional[Dict[str, str]]], Optional[Dict[str, str]]]] = None,
                 core: Dict = CORE):
        super().__init__(config, core)

        # insert copasi process model config
        model_source = self.config['model'].get('model_source') or self.config.get('sbml_fp')
        assert model_source is not None, 'You must specify a model source of either a valid biomodel id or model filepath.'
        model_changes = self.config['model'].get('model_changes', {})
        self.model_changes = {} if model_changes is None else model_changes

        # Option A:
        if '/' in model_source:
            self.copasi_model_object = load_model(model_source)
            print('found a filepath')

        # Option C:
        else:
            if not self.model_changes:
                raise AttributeError(
                    """You must pass a source of model changes specifying params, reactions, 
                        species or all three if starting from an empty model.""")
            model_units = self.config['model'].get('model_units', {})
            self.copasi_model_object = new_model(
                name='CopasiProcess TimeCourseModel',
                **model_units)

        # handle context of species output
        context_type = self.config.get('species_context', 'concentrations')
        self.species_context_key = f'floating_species_{context_type}'
        self.use_counts = 'concentrations' in context_type

        # Get a list of reactions
        self._set_reaction_changes()
        reactions = get_reactions(model=self.copasi_model_object)
        self.reaction_list = reactions.index.tolist() if reactions is not None else []
        reaction_data = get_reactions(model=self.copasi_model_object)['scheme']
        reaction_schemas = reaction_data.values.tolist()
        reaction_names = reaction_data.index.tolist()
        self.reactions = dict(zip(reaction_names, reaction_schemas))

        # Get the species (floating only)  TODO: add boundary species
        self._set_species_changes()
        species_data = get_species(model=self.copasi_model_object)
        self.floating_species_list = species_data.index.tolist()
        self.floating_species_initial = species_data.particle_number.tolist() \
            if self.use_counts else species_data.concentration.tolist()

        # Get the list of parameters and their values (it is possible to run a model without any parameters)
        self._set_global_param_changes()
        model_parameters = get_parameters(model=self.copasi_model_object)
        self.model_parameters_list = model_parameters.index.tolist() \
            if isinstance(model_parameters, DataFrame) else []
        self.model_parameters_values = model_parameters.initial_value.tolist() \
            if isinstance(model_parameters, DataFrame) else []

        # Get a list of compartments
        self.compartments_list = get_compartments(model=self.copasi_model_object).index.tolist()

        # ----SOLVER: Get the solver (defaults to deterministic)
        self.method = self.config['method']

    def initial_state(self):
        # keep in mind that a valid simulation may not have global parameters
        model_parameters_dict = dict(
            zip(self.model_parameters_list, self.model_parameters_values))

        floating_species_dict = dict(
            zip(self.floating_species_list, self.floating_species_initial))

        return {
            'time': 0.0,
            'model_parameters': model_parameters_dict,
            self.species_context_key: floating_species_dict,
            'reactions': self.reactions
        }

    def inputs(self):
        # dependent on species context set in self.config
        floating_species_type = {
            species_id: {
                '_type': 'float',
                '_apply': 'set'}
            for species_id in self.floating_species_list
        }

        model_params_type = {
            param_id: {
                '_type': 'float',
                '_apply': 'set'}
            for param_id in self.model_parameters_list
        }

        reactions_type = {
            reaction_id: 'string'
            for reaction_id in self.reaction_list
        }

        return {
            'time': 'float',
            self.species_context_key: floating_species_type,
            'model_parameters': model_params_type,
            'reactions': 'tree[string]'
        }

    def outputs(self):
        floating_species_type = {
            species_id: {
                '_type': 'float',
                '_apply': 'set'}
            for species_id in self.floating_species_list
        }
        reactions_type = {
            reaction_id: 'string'
            for reaction_id in self.reaction_list
        }
        return {
            'time': 'float',
            self.species_context_key: floating_species_type,
            'reactions': 'tree[string]'
        }

    def update(self, inputs, interval):
        # set copasi values according to what is passed in states for concentrations
        for cat_id, value in inputs[self.species_context_key].items():
            set_type = 'particle_number' if 'counts' in self.species_context_key else 'concentration'
            species_config = {
                'name': cat_id,
                'model': self.copasi_model_object,
                set_type: value}
            set_species(**species_config)

        # run model for "interval" length; we only want the state at the end
        timecourse = run_time_course(
            start_time=inputs['time'],
            duration=interval,
            update_model=True,
            model=self.copasi_model_object,
            method=self.method)

        # extract end values of concentrations from the model and set them in results
        results = {'reactions': self.reactions}
        if self.use_counts:
            results[self.species_context_key] = {
                mol_id: float(get_species(
                    name=mol_id,
                    exact=True,
                    model=self.copasi_model_object
                ).particle_number[0])
                for mol_id in self.floating_species_list}
        else:
            results[self.species_context_key] = {
                mol_id: float(get_species(
                    name=mol_id,
                    exact=True,
                    model=self.copasi_model_object
                ).concentration[0])
                for mol_id in self.floating_species_list}

        return results

    def _set_reaction_changes(self):
        # ----REACTIONS: set reactions
        existing_reactions = get_reactions(model=self.copasi_model_object)
        existing_reaction_names = existing_reactions.index.tolist() if existing_reactions is not None else []
        reaction_changes = self.model_changes.get('reaction_changes', [])
        if reaction_changes:
            for reaction_change in reaction_changes:
                reaction_name: str = reaction_change['reaction_name']
                param_changes: list[dict[str, float]] = reaction_change['parameter_changes']
                scheme_change: str = reaction_change.get('reaction_scheme')
                # handle changes to existing reactions
                if param_changes:
                    for param_name, param_change_val in param_changes:
                        set_reaction_parameters(param_name, value=param_change_val, model=self.copasi_model_object)
                if scheme_change:
                    set_reaction(name=reaction_name, scheme=scheme_change, model=self.copasi_model_object)
                # handle new reactions
                if reaction_name not in existing_reaction_names and scheme_change:
                    add_reaction(reaction_name, scheme_change, model=self.copasi_model_object)

    def _set_species_changes(self):
        # ----SPECS: set species changes
        species_changes = self.model_changes.get('species_changes', [])
        if species_changes:
            for species_change in species_changes:
                if isinstance(species_change, dict):
                    species_name = species_change.pop('name')
                    changes_to_apply = {}
                    for spec_param_type, spec_param_value in species_change.items():
                        if spec_param_value:
                            changes_to_apply[spec_param_type] = spec_param_value
                    set_species(**changes_to_apply, model=self.copasi_model_object)

    def _set_global_param_changes(self):
        # ----GLOBAL PARAMS: set global parameter changes
        global_parameter_changes = self.model_changes.get('global_parameter_changes', [])
        if global_parameter_changes:
            for param_change in global_parameter_changes:
                param_name = param_change.pop('name')
                for param_type, param_value in param_change.items():
                    if not param_value:
                        param_change.pop(param_type)
                    # handle changes to existing params
                    set_parameters(name=param_name, **param_change, model=self.copasi_model_object)
                    # set new params
                    global_params = get_parameters(model=self.copasi_model_object)
                    if global_params:
                        existing_global_parameters = global_params.index
                        if param_name not in existing_global_parameters:
                            assert param_change.get('initial_concentration') is not None, "You must pass an initial_concentration value if adding a new global parameter."
                            add_parameter(name=param_name, **param_change, model=self.copasi_model_object)

In [3]:
CORE.process_registry.register('fba', CobraProcess)
CORE.process_registry.register('copasi', CopasiProcess)

CORE.process_registry.registry

{'console-emitter': process_bigraph.composite.ConsoleEmitter,
 'ram-emitter': process_bigraph.composite.RAMEmitter,
 'composite': process_bigraph.composite.Composite,
 'fba': __main__.CobraProcess,
 'copasi': __main__.CopasiProcess}

In [4]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

cobra_process = CobraProcess(config={'model': {'model_source': model_fp}}, core=CORE)

model = cobra_process.model
copasi_process = CopasiProcess(config={'model': {'model_source': model_fp}}, core=CORE)
# copasi_process.outputs()

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

found a filepath


In [8]:
reactions = [r.name for r in model.reactions]

reactions

['degradation of LacI transcripts',
 'degradation of TetR transcripts',
 'degradation of CI transcripts',
 'translation of LacI',
 'translation of TetR',
 'translation of CI',
 'degradation of LacI',
 'degradation of TetR',
 'degradation of CI',
 'transcription of LacI',
 'transcription of TetR',
 'transcription of CI']

In [5]:
# model = cobra_process.model
copasi_spec = {
    'ode': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model': {
                'model_source': model_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'reactions': ['reactions_store']
        }
    }
}

fba_spec = {
    'fba': {
        '_type': 'process',
        'address': 'local:fba',
        'config': {'model': {'model_source': model_fp}},
        'inputs': {
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'reactions': ['reactions_store'],
        },
        'outputs': {
            'substrates': ['substrates_store'],
        }
    }
}

emitter_spec = {
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species_concentrations': 'tree[float]',
                'substrates': 'tree[float]',
                'time': 'float'
            }
        },
        'inputs': {
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'substrates': ['substrates_store'],
            'time': ['time_store']
        }
    }
}

In [6]:
spec = copasi_spec.copy()
spec.update(fba_spec)
spec.update(emitter_spec)

spec

{'ode': {'_type': 'process',
  'address': 'local:copasi',
  'config': {'model': {'model_source': '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'}},
  'inputs': {'time': ['time_store'],
   'floating_species_concentrations': ['floating_species_concentrations_store'],
   'model_parameters': ['model_parameters_store'],
   'reactions': ['reactions_store']},
  'outputs': {'time': ['time_store'],
   'floating_species_concentrations': ['floating_species_concentrations_store'],
   'reactions': ['reactions_store']}},
 'fba': {'_type': 'process',
  'address': 'local:fba',
  'config': {'model': {'model_source': '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'}},
  'inputs': {'floating_species_concentrations': ['floating_species_concentrations_store'],
   'reactions': ['reactions_store']},
  'output

In [7]:
comp = Composite(config={'state': spec}, core=CORE)

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

found a filepath


NameError: name 'solution' is not defined

In [48]:
comp.run(10)

/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/ipykernel_21224/2868703094.py:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mol_id: float(get_species(


In [49]:
r = comp.gather_results()

r

{('emitter',): [{'floating_species_concentrations': {'LacI protein': 0.0,
    'TetR protein': 0.0,
    'cI protein': 0.0,
    'LacI mRNA': 0.0,
    'TetR mRNA': 20.0,
    'cI mRNA': 0.0},
   'substrates': {},
   'time': 0.0},
  {'floating_species_concentrations': {'LacI protein': 81.44046884923394,
    'TetR protein': 188.3821831030562,
    'cI protein': 42.64131125320374,
    'LacI mRNA': 19.90344166947679,
    'TetR mRNA': 50.61558010446934,
    'cI mRNA': 7.491001903157758},
   'substrates': {'LacI mRNA': 0.0,
    'TetR mRNA': 0.0,
    'cI mRNA': 0.0,
    'LacI protein': 0.0,
    'TetR protein': 0.0,
    'cI protein': 0.0},
   'time': 0.0},
  {'floating_species_concentrations': {'LacI protein': 299.9792524293013,
    'TetR protein': 546.4092678039393,
    'cI protein': 127.23014070279017,
    'LacI mRNA': 41.13697667160224,
    'TetR mRNA': 74.22450737953378,
    'cI mRNA': 13.357268321541767},
   'substrates': {'LacI mRNA': 0.0,
    'TetR mRNA': 0.0,
    'cI mRNA': 0.0,
    'LacI p

In [11]:
copasi_process.floating_species_list == cobra_process.metabolite_names

True

In [64]:
from basico import load_model, run_timecourse, set_parameter_value
import cobra
import numpy as np


# Function to modify dynamic bounds
def add_dynamic_bounds(model, y):
    """Use external concentrations to bound the uptake flux of glucose."""
    biomass, glucose = y  # expand the boundary species

    # Example dynamic bound based on glucose concentration
    glucose_max_import = -10 * glucose / (5 + glucose)  # Modify based on glucose availability
    reaction_id = 'EX_glc__D_e'  # Replace with the actual glucose exchange reaction ID in your model

    # Apply the dynamic bound to glucose uptake in the model
    model.reactions.get_by_id(reaction_id).lower_bound = glucose_max_import


# Function to run dynamic FBA at each step
def dynamic_fba_step(model, y):
    """Calculate the dynamic FBA fluxes at a given timepoint."""
    biomass, glucose = y

    # Add dynamic bounds to the model based on the current species concentration
    add_dynamic_bounds(model, y)

    # Perform FBA optimization
    solution = model.optimize()

    # Extract the fluxes (reaction rates) of interest
    biomass_flux = solution.fluxes['Biomass_Ecoli_core']  # Replace with your model's biomass reaction ID
    glucose_flux = solution.fluxes['EX_glc__D_e']  # Replace with your model's glucose uptake reaction

    # Return these fluxes as the rate of change of biomass and glucose
    return [biomass_flux, glucose_flux]


# ODE function using dynamic FBA results
def dynamic_ode_with_fba(t, y, model):
    """
    Custom ODE system using the dynamic FBA results for each timepoint.
    """
    # Calculate the fluxes (reaction rates) for the given concentrations at time t
    return dynamic_fba_step(model, y)


# Function to run the entire dynamic simulation using basico
def run_dynamic_fba_with_basico(sbml_file, y0, t_span, t_eval):
    """
    Runs a dynamic FBA system using basico's time course simulation.
    """
    # Load the model using basico (assuming it is already in SBML format)
    model = load_model(sbml_file)

    # Set initial conditions and time span
    initial_conditions = y0
    timespan = t_span
    time_eval = t_eval

    # Run the timecourse with basico (this will simulate the ODE dynamics)
    result = run_timecourse(model, start=timespan[0], end=timespan[1], steps=len(time_eval))

    # Iterate over each step and adjust bounds dynamically based on ODE results
    for i, timepoint in enumerate(time_eval):
        # Get current concentrations from the basico simulation output
        concentrations = result['floating_species_concentrations']  # Adjust this based on basico's output format
        biomass = concentrations['Biomass']  # Replace with the actual species name in your model
        glucose = concentrations['Glucose']  # Replace with the actual species name in your model

        # Run dynamic FBA to get flux updates at each timepoint
        fluxes = dynamic_ode_with_fba(timepoint, [biomass, glucose], model)

        # Optionally, store these fluxes or update the model for visualization or further analysis

    # Return the final result (fluxes and timecourse)
    return result


# Example of running the dynamic FBA with basico's timecourse
if __name__ == '__main__':
    # Example SBML file (replace with actual file path)
    sbml_file = "path_to_model.xml"

    # Set initial conditions [biomass, glucose]
    y0 = [0.1, 10.0]  # Initial values for biomass and glucose

    # Time span and evaluation points
    t_span = (0, 15)  # Start and end time
    t_eval = np.linspace(0, 15, 100)  # Points to evaluate

    # Run the dynamic FBA with basico
    result = run_dynamic_fba_with_basico(sbml_file, y0, t_span, t_eval)

    # Output the result (biomass, glucose concentrations over time)
    print("Result from basico timecourse:", result)


PX
==
Formula: None

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
PY
==
Formula: None

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
PZ
==
Formula: None

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
X
=
Formula: None

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
Y
=
Formul

In [42]:
copasi_process.update(copasi_process.initial_state(), 1)

/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/ipykernel_5473/1817879026.py:178: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mol_id: float(get_species(


{'reactions': ['degradation of LacI transcripts',
  'degradation of TetR transcripts',
  'degradation of CI transcripts',
  'translation of LacI',
  'translation of TetR',
  'translation of CI',
  'degradation of LacI',
  'degradation of TetR',
  'degradation of CI',
  'transcription of LacI',
  'transcription of TetR',
  'transcription of CI'],
 'floating_species_concentrations': {'LacI protein': 81.44046884923394,
  'TetR protein': 188.3821831030562,
  'cI protein': 42.64131125320374,
  'LacI mRNA': 19.90344166947679,
  'TetR mRNA': 30.615580104469338,
  'cI mRNA': 7.491001903157758}}

In [32]:
# Cobra process inputs: time, floating_species_concentrations, reactions
# 1. constructor creates empty model using: Upper and Lower bounds (defaults)
# 2. In update:
#   - for reaction in inputs['reactions'].keys(): create a new reaction with that name 

degradation of LacI transcripts
degradation of TetR transcripts
degradation of CI transcripts
translation of LacI
translation of TetR
translation of CI
degradation of LacI
degradation of TetR
degradation of CI
transcription of LacI
transcription of TetR
transcription of CI


In [21]:
def automate_cobra_model_creation(reactions_dict, species_names):
    # Create a new COBRA model
    model = Model("automated_cobra_model")
    reactions = list(reactions_dict.values())
    reaction_names = list(reactions_dict.keys())
    
    # Step 1: Create Metabolite objects for all species
    metabolite_dict = {}
    for species in species_names:
        metabolite = Metabolite(
            species.replace(" ", "_").replace('"', ''),  # Ensure the metabolite ID is valid
            name=species,
            compartment='c'  # Assuming all species are in the cytosol
        )
        metabolite_dict[species] = metabolite  # Add to dictionary for easy reference

    # Step 2: Create Reaction objects and add Metabolites
    for i, reaction_scheme in enumerate(reactions):
        reaction_name = reaction_names[i]
        reaction_id = reaction_name.replace(" ", "_")
        reaction = Reaction(reaction_id)  # Create a reaction with the ID
        
        # Parse the reaction scheme to identify substrates and products
        scheme_str = reaction_scheme
        reactants, products = scheme_str.split("->")
        
        # Initialize the dictionary for metabolites with their stoichiometry
        metabolites_to_add = {}

        # Handle Reactants
        reactant_list = reactants.split(";")
        for reactant in reactant_list:
            reactant = reactant.strip().replace('"', '')  # Clean up string
            if reactant in metabolite_dict:
                metabolites_to_add[metabolite_dict[reactant]] = -1.0  # Reactants are consumed (-1)

        # Handle Products
        product_list = products.split(";")
        for product in product_list:
            product = product.strip().replace('"', '')  # Clean up string
            if product in metabolite_dict:
                metabolites_to_add[metabolite_dict[product]] = 1.0  # Products are produced (+1)

        # Add metabolites to the reaction
        reaction.add_metabolites(metabolites_to_add)

        # Add the reaction to the model
        model.add_reactions([reaction])

    return model


model = automate_cobra_model_creation(reactions, species_names)

NameError: name 'reactions' is not defined

In [14]:
solution = model.optimize()

In [37]:
rs = model.metabolites[0].id
rs

'LacI_mRNA'

In [1]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'
copasi_spec = {
    'ode': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model': {
                'model_source': model_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
        }
    }
}

# fba_spec = {
#     'fba': {
#         '_type': 'process',
#         'address': 'local:fba',
#         'config': {},
#         'inputs': {
#             'substrates': ['floating_species_concentrations_store'],
#         },
#         'outputs': {
#             'substrates': ['substrates_store'],
#         }
#     }
# }

fem_spec = {}

emitter_spec = {
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'substrates': 'tree[float]',
                'time': 'float',
                # 'floating_species_concentrations': 'tree[float]'
            }
        },
        'inputs': {
            'substrates': ['substrates_store'],
            'time': ['time_store'],
            # 'floating_species_concentrations': ['floating_species_concentrations_store']
        }
    }
}

In [ ]:
from biosimulators_processes.processes.cobra_process import run_dfba_spatial

run_dfba_spatial()



Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.


In [6]:
process.model.boundary

[<Reaction Reaction1 at 0x3485fe590>,
 <Reaction Reaction2 at 0x10b9506a0>,
 <Reaction Reaction3 at 0x3485ff700>,
 <Reaction Reaction4 at 0x3485fea40>,
 <Reaction Reaction5 at 0x3485fd990>,
 <Reaction Reaction6 at 0x3485fe9e0>,
 <Reaction Reaction7 at 0x3485ff340>,
 <Reaction Reaction8 at 0x3485ff880>,
 <Reaction Reaction9 at 0x3485ff2e0>,
 <Reaction Reaction10 at 0x3485ffe50>,
 <Reaction Reaction11 at 0x3485fee60>,
 <Reaction Reaction12 at 0x3485ff0a0>]

In [11]:
from basico import *
import pandas as pd

model = load_model(model_fp)

# Run the ODE simulation using basico
timepoints = [0, 10, 20, 30, 40, 50]  # Time points for the simulation
result = run_time_course(model, timepoints)

# Convert to DataFrame for easier manipulation
concentration_df = pd.DataFrame(result['concentrations'], columns=result['species_ids'])
concentration_df['time'] = result['timepoints']
print(concentration_df)

TypeError: float() argument must be a string or a real number, not 'CDataModel'

In [140]:
from basico import *
from cobra import Model
from cobra.io import read_sbml_model
import pandas as pd
import numpy as np
import logging 
from tqdm import tqdm
from scipy.integrate import solve_ivp
import cobra.util.solver


logging.getLogger('cobra').setLevel(logging.ERROR)
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

def generate_timecourse(model_fp=model_fp, start=0, stop=10, steps=100):
    model = load_model(model_fp)
    outputs = get_species(model=model).index.tolist()
    t = np.linspace(0, 10, 1000)
    tc = run_time_course_with_output(outputs, t[-1], values=t, model=model)
    return tc 


def generate_reaction_mappings(cobra_model, output_names):
    mappings = []
    for reaction in cobra_model.reactions:
        for name in output_names:
            rxn = [r.lower() for r in reaction.name.split(" ")]
            obs_name = name.split(" ")[0].lower()
            obs_type = name.split(" ")[-1]
            if obs_name in rxn:
                mapping = {}
                if "transcription" in rxn and obs_type == "mRNA":
                    mapping = {name: reaction.name}
                elif "translation" in rxn and obs_type == "protein":
                    mapping = {name: reaction.name}
                elif "degradation" in rxn:
                    if "transcripts" in rxn and obs_type == "mRNA":
                        mapping = {name: reaction.name}
                    elif "transcripts" not in rxn and obs_type == "protein":
                        mapping = {name: reaction.name}
                if mapping:
                    mappings.append(mapping)
    return mappings


def apply_ode_results_to_fba(time_point, ode_results, cobra_model, mappings):
    """
    Apply ODE results to COBRApy model reactions based on mappings.
    Args:
        time_point: Current time step in the ODE simulation
        ode_results: Time-series DataFrame with concentration data
        cobra_model: COBRApy model
        mappings: List of mappings between ODE outputs and COBRA reactions
    """
    for mapping in mappings:
        for ode_species, cobra_reaction in mapping.items():
            # Extract the concentration at the current time point
            concentration = ode_results.loc[time_point, ode_species]
            
            # Apply this concentration to the relevant reaction in COBRA model
            # For example, if it is a degradation reaction, you can set a lower bound
            reaction = cobra_model.reactions.get_by_id(cobra_reaction)
            
            # Modify reaction bounds based on concentration
            if 'degradation' in cobra_reaction:
                reaction.lower_bound = -concentration  # Assuming a negative flux for degradation
            else:
                # Example: You could modify the upper bound or other parameters
                reaction.upper_bound = concentration
                

result = generate_timecourse()
output_names = result.columns.tolist()
time_points = result.index.tolist()
cobra_model = read_sbml_model(model_fp)
reaction_mappings = generate_reaction_mappings(cobra_model, output_names)

ModuleNotFoundError: No module named 'tqdm'

In [138]:

output_names

['LacI protein',
 'TetR protein',
 'cI protein',
 'LacI mRNA',
 'TetR mRNA',
 'cI mRNA']

In [37]:
tc.keys()

dict_keys(['LacI protein', 'TetR protein', 'cI protein', 'LacI mRNA', 'TetR mRNA', 'cI mRNA'])

In [38]:
outputs

['LacI protein',
 'TetR protein',
 'cI protein',
 'LacI mRNA',
 'TetR mRNA',
 'cI mRNA']

In [115]:
def generate_reaction_mappings(cobra_model, output_names):
    mappings = []
    for reaction in cobra_model.reactions:
        for name in output_names:
            rxn = [r.lower() for r in reaction.name.split(" ")]
            obs_name = name.split(" ")[0].lower()
            obs_type = name.split(" ")[-1]
            if obs_name in rxn:
                mapping = {}
                if "transcription" in rxn and obs_type == "mRNA":
                    mapping = {name: reaction.name}
                elif "translation" in rxn and obs_type == "protein":
                    mapping = {name: reaction.name}
                elif "degradation" in rxn:
                    if "transcripts" in rxn and obs_type == "mRNA":
                        mapping = {name: reaction.name}
                    elif "transcripts" not in rxn and obs_type == "protein":
                        mapping = {name: reaction.name}
                if mapping:
                    mappings.append(mapping)
    return mappings

In [117]:
len(_generate_reaction_mappings(cobra_model, outputs))

12

In [103]:
[r.name for r in cobra_model.reactions]

['degradation of LacI transcripts',
 'degradation of TetR transcripts',
 'degradation of CI transcripts',
 'translation of LacI',
 'translation of TetR',
 'translation of CI',
 'degradation of LacI',
 'degradation of TetR',
 'degradation of CI',
 'transcription of LacI',
 'transcription of TetR',
 'transcription of CI']

In [104]:
outputs

['LacI protein',
 'TetR protein',
 'cI protein',
 'LacI mRNA',
 'TetR mRNA',
 'cI mRNA']

In [107]:
'LacI'.lower()

'laci'

In [108]:
"mRNA".lower()

'mrna'

In [110]:
'degradation of CI transcripts'.lower()

'degradation of ci transcripts'